# Metriken
## M1 Downstram Classification Task
Hier werde ich ein einfaches LSTM aufbauen und verwenden. Die gelernten word vectors werden im embedding layer eingesetzt und dann wird die accuracy als Metrik verwendet.
## M2 Clustering by Subsystem
+ compile a list of 3-5 subsystems and collect 10-20 words that belong to the subsystem
+ build a clustering algorithm in the embedding space based on the cosine-distance


## M3 Word Analogy

a:b::c:d



## M4 Word Similarity List
Hier werde ich Norberts Liste verwenden und dann die cosine-Distanz der word embeddings mit der gemittelten Similarity korrelieren (PEARSON correlation oder sowas? )




In [1]:
import pickle

In [ ]:
# M1 LSTM

In [10]:
# M2 Clustering by subsystem

file = open("/home/seb/Capstone Project/Data/CRQ/CRQ_preprocessed.pkl", 'rb')
crq = pickle.load(file)

engine = ['motor', 'einspritzung', 'zündung']
transmission = ['getriebe', "gang", "wählhebel", "fahrstufe", "schlupf", "kupplung"]
control_unit = ["funktion", "applikation", "regler", ]
vehicle=["can", "kombiinstrument", 'flexray', ]
exhaust

print(crq[130:150])

[['modul', 'hvbattsof', 'werte', 'hv', 'batterie', 'aufbereitet', 'audilib', 'verfügung', 'gestellt', 'gibt', 'modul', 'funktion', 'relativen', 'soc', 'benötigten', 'grenzen', 'bereit', 'stellt', 'problem', 'verschiedene', 'anforderer', 'projekte', 'unterschiedliche', 'lösungen', 'balkon', 'ursprüngliche', 'funktion', 'angehängt', 'worden', 'folgen', 'logikteile', 'doppelt', 'vorkommen', 'ungewollt', 'ausschließen', 'funktion', 'schwer', 'verstehen', 'dadurch', 'erhöhte', 'gefahr', 'falschen', 'applikation', 'mehrfachverwendung', 'applikationsgrößen', 'besteht', 'möglichkeit', 'funktionen', 'modul', 'falsch', 'bedaten', 'z', 'b', 'tiefentladeschutz', 'umsetzung', 'modular', 'bsp', 'tiefentladeschutzkonzept', 'vernetzung', 'möglich', 'strukturierte', 'modularisierte', 'darstellung', 'funktionalität', 'sicherstellung', 'projektanforderung', 'konfiguration', 'funktion', 'applikation', 'entkoppelung', 'applikationsgrößen', 'einzelnen', 'funktionalitäten'], ['test'], ['zustand', 'einschalte

In [ ]:
# M3 Word Analogy

In [ ]:
# M4 Word similarity